In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

/opt/tljh/user/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## Setting up the instance

### Screenshot

![](https://github.com/UBC-MDS/web_cloud_comp_group2/raw/main/notebooks/img/image_1.png)

## Setting up JupyterHub

### Screenshot

![](https://github.com/UBC-MDS/web_cloud_comp_group2/raw/main/notebooks/img/image_2.png)

## Setting up the server

### Screenshot

![](https://github.com/UBC-MDS/web_cloud_comp_group2/raw/main/notebooks/img/image_3.png)

## Setting up the S3 bucket and moving the data

### Screenshot

![](https://github.com/UBC-MDS/web_cloud_comp_group2/raw/main/notebooks/img/image_4.png)

## Wrangling the data

In [ ]:
# Necessary metadata
article_id = 14226968  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "/srv/data/my_shared_data_folder/"

In [ ]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

In [ ]:
files_to_dl = ["combined_model_data_parti.parquet.zip"]  ## Please download the partitioned 
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [ ]:
with zipfile.ZipFile(os.path.join(output_directory, "combined_model_data_parti.parquet.zip"), 'r') as f:
    f.extractall(output_directory)

In [2]:
# Passing your credentials 
## IMPORTANT: make sure you dont include you secret and key when submitting the notebook
aws_credentials ={"key": "","secret": ""} ## dont include you secret and key when submitting the notebook
df = pd.read_parquet("s3://mds-s3-student1/combined_model_data_parti.parquet", storage_options=aws_credentials)
df_syd_observed = pd.read_csv("s3://mds-s3-student1/observed_daily_rainfall_SYD.csv", storage_options=aws_credentials, index_col='time')

In [3]:
df.shape

(62513863, 7)

In [4]:
df_syd_observed.shape

(46020, 1)

In [5]:
syd_lat = -33.86
syd_lon = 151.21

df = df.query("lat_min <= @syd_lat <= lat_max ").query("lon_min <= @syd_lon <= lon_max ")
df.index = df.time
df = df.drop(columns=['lat_min', 'lat_max', 'lon_min', 'lon_max', 'time'])
df = df.pivot(columns='model', values='rain (mm/day)')
df = df.resample('D').mean()
df['observed'] = df_syd_observed['rain (mm/day)'].to_numpy()

In [6]:
df

model       ACCESS-CM2  ACCESS-ESM1-5  AWI-ESM-1-1-LR   BCC-CSM2-MR  \
time                                                                  
1889-01-01    0.040427       1.814552    3.557934e+01  4.268112e+00   
1889-01-02    0.073777       0.303965    4.596520e+00  1.190141e+00   
1889-01-03    0.232656       0.019976    5.927467e+00  1.003845e-09   
1889-01-04    0.911319      13.623777    8.029624e+00  8.225225e-02   
1889-01-05    0.698013       0.021048    2.132686e+00  2.496841e+00   
...                ...            ...             ...           ...   
2014-12-27    0.033748       0.123476    1.451179e+00  3.852845e+01   
2014-12-28    0.094198       2.645496    4.249335e+01  5.833801e-01   
2014-12-29    0.005964       3.041667    2.898325e+00  9.359547e-02   
2014-12-30    0.000028       1.131412    2.516381e-01  1.715028e-01   
2014-12-31    0.532747       2.370896    1.047835e-13  4.437736e+00   

model           BCC-ESM1  CMCC-CM2-HR4  CMCC-CM2-SR5  CMCC-ESM2    CanESM5  \
time                                                                         
1889-01-01  1.107466e-03  1.141054e+01  3.322009e-08   2.668800   1.321215   
1889-01-02  1.015323e-04  4.014984e+00  1.312700e+00   0.946211   2.788724   
1889-01-03  1.760345e-05  9.660565e+00  9.103720e+00   0.431999   0.003672   
1889-01-04  1.808932e-01  3.951528e+00  1.317160e+01   0.368693   0.013578   
1889-01-05  4.708019e-09  2.766362e+00  1.822940e+01   0.339267   0.002468   
...                  ...           ...           ...        ...        ...   
2014-12-27  2.061717e-03  8.179260e-09  1.171263e-02   0.090786  59.895053   
2014-12-28  5.939502e-09  8.146937e-01  4.938899e-01   0.000000   0.512632   
2014-12-29  2.000051e-08  2.532205e-01  1.306046e+00   0.000002  37.169669   
2014-12-30  7.191735e-05  8.169252e-02  1.722262e-01   0.788577   7.361246   
2014-12-31  2.863683e-01  6.343592e+00  6.368303e-01   0.442130   0.306608   

model       EC-Earth3-Veg-LR  ...  MPI-ESM-1-2-HAM  MPI-ESM1-2-HR  \
time                          ...                                   
1889-01-01          1.515293  ...     4.244226e-13   1.390174e-13   
1889-01-02          4.771375  ...     4.409552e+00   1.222283e-01   
1889-01-03          4.233980  ...     2.269300e-01   3.762301e-01   
1889-01-04         15.252495  ...     2.344586e-02   4.214019e-01   
1889-01-05         11.920356  ...     4.270161e-13   1.879692e-01   
...                      ...  ...              ...            ...   
2014-12-27          5.071783  ...     4.726998e-13   1.326889e-01   
2014-12-28          1.578188  ...     4.609420e-13   1.644482e+00   
2014-12-29          1.565885  ...     2.016156e+01   1.506439e+00   
2014-12-30          0.025749  ...     9.420543e+00   6.242895e+00   
2014-12-31          0.106525  ...     1.031899e+01   4.765813e+01   

model       MPI-ESM1-2-LR    MRI-ESM2-0         NESM3    NorESM2-LM  \
time                                                                  
1889-01-01   6.537884e-05  3.445495e-06  1.576096e+01  4.759651e-05   
1889-01-02   1.049131e-13  4.791993e-09  3.675510e-01  4.350863e-01   
1889-01-03   9.758706e-14  6.912302e-01  1.562869e-01  9.561101e+00   
1889-01-04   7.060915e-03  3.835721e-02  2.472226e-07  5.301038e-01   
1889-01-05   4.504985e+00  3.506923e-07  1.949792e-13  1.460928e-10   
...                   ...           ...           ...           ...   
2014-12-27   1.827857e+00  6.912632e-03  2.171327e-03  1.620489e+00   
2014-12-28   7.242920e-01  2.836752e-03  1.344768e+01  2.391159e+00   
2014-12-29   1.049481e-01  8.137182e+00  2.547820e+01  1.987695e-12   
2014-12-30   1.245115e-01  9.305263e-03  4.192948e+00  2.150346e+00   
2014-12-31   3.274323e-01  6.854985e-11  2.067847e+00  2.349716e+01   

model       NorESM2-MM  SAM0-UNICON    TaiESM1   observed  
time                                                       
1889-01-01    2.451075     0.221324   2.257933   0.006612  
1889-01-02    0.477231     3.757179   2.287381   0.090422  

In [7]:
df.to_csv('s3://mds-s3-student1/output/ml_data_SYD.csv', storage_options=aws_credentials)

### Screenshot

![](https://github.com/UBC-MDS/web_cloud_comp_group2/raw/main/notebooks/img/image_5.png)